In [190]:
import requests
import bs4
from bs4 import BeautifulSoup as bs
import re
import time
import json

# List of That 70's Show scripts

In [22]:
url = 'https://www.springfieldspringfield.co.uk/episode_scripts.php?tv-show=that-70s-show'
root_url = url.split('episode_scripts',1)[0]

In [7]:
page = requests.get(url)
html = bs(page.text, 'html.parser')

In [10]:
episodes = html.find_all('a', class_='season-episode-title')

## Get the script for each available (if available)

In [ ]:
all_episodes = {}
num_episodes = len(episodes)
regex = re.compile(r'(s\d{2})(e\d{2})')
for n,episode in enumerate(episodes):
    try:
        _url = root_url + episode.get('href')
        _page = requests.get(_url)
        _html = bs(_page.text, 'html.parser')
        txt = _html.find('div', class_="scrolling-script-container")      
        title = _html.find('h3').text.strip()
        print("({n}/{total}): {title}".format(n=n+1, total=num_episodes, title=title))

        # Store each line of the script as an entry in a list
        script = [line.strip() for line in txt.contents if type(line)==bs4.element.NavigableString]

        # Store the episode in JSON format    
        if script:

            # Get season and episode number
            title_info = _html.find('h1').text.strip()
            se_and_ep = regex.findall(title_info)[0]
            num_se = int(se_and_ep[0].split('s',1)[-1])
            num_ep = int(se_and_ep[1].split('e',1)[-1])        
            data = {'title': title, 'season': num_se, 'episode': num_ep, 'script': script, 'raw': str(txt)}

            if num_se in all_episodes:
                all_episodes[num_se].append(data)
            else:
                all_episodes[num_se] = [data]            
    except Exception as e:
        print("Error: {e}".format(e=e))
        time.sleep(30)
        
    if n%20==0:
        filename = 'That70sData_v{}.json'.format(n)
        with open(filename, 'w') as outfile:
            json.dump(all_episodes, outfile)            

    time.sleep(10)
    
# Save all of the data
with open('That70sData.json', 'w') as outfile:
    json.dump(all_episodes, outfile)

(1/200): That '70s Pilot
(2/200): Eric's Birthday
(3/200): Streaking
(4/200): Battle of the Sexists
(5/200): Eric's Burger Job
(6/200): The Keg
(7/200): That Disco Episode
(8/200): Drive-In
(9/200): Thanksgiving
(10/200): Sunday, Bloody Sunday
(11/200): Eric's Buddy
(12/200): The Best Christmas Ever
(13/200): Ski Trip
(14/200): Stolen Car
(15/200): That Wrestling Show
(16/200): The First Date
(17/200): The Pill
(18/200): The Career Day
(19/200): Prom Night
(20/200): A New Hope
(21/200): Water Tower
(22/200): Punk Chick
(23/200): Grandma's Dead
(24/200): Hyde Moves In
(25/200): The Good Son
(26/200): Garage Sale
(27/200): Red's Last Day
(28/200): The Velvet Rope
(29/200): Laurie and the Professor
(30/200): Halloween
(31/200): Vanstock
(32/200): I Love Cake
(33/200): Sleepover
(34/200): Eric Gets Suspended
(35/200): Red's Birthday
(36/200): Laurie Moves Out
(37/200): Eric's Stash
(38/200): Hunting
(39/200): Red's New Job
(40/200): Burning Down the House
(41/200): The First Time
(42/200):

In [140]:
all_episodes[0][0].keys()

dict_keys(['title', 'season', 'episode', 'script'])

In [141]:
all_episodes[0][0]

{'title': "That '70s Pilot",
 'season': 1,
 'episode': 1,
 'script': ['Eric.',
  'It is time.',
  "Why don't you do it? - It's your house.",
  '- Your house.',
  'Listen to them up there.',
  'The party has reached critical mass.',
  'In 10 minutes, there will be no more beer opportunities.',
  "If my dad catches me copping beers, he'll kill me.",
  "I'm willing to take that risk.",
  "Don't worry about it.",
  'Just remain calm, keep moving.',
  "And above all, don't get sucked into my dad's hair.",
  "- What's wrong with your dad's hair? - Just don't look at it.",
  'And, Eric, cold.',
  'Definitely cold.',
  'Young and beautiful Someday your looks will be gone Watch it, Eric! Hot pizza rolls.',
  'Coming through.',
  'Hot, hot.',
  'Kitty, where are you? Okay, take two.',
  "There's plenty.",
  "There's plenty coming.",
  'Is everybody good? I know.',
  'I know.',
  'Vienna sausages are so versatile.',
  'Hi, there, Eric.',
  'Mr.',
  'Pinciotti.',
  "So, Eric, how do you like Bob's

---
# Analysis

### Load the script data

In [ ]:
filename = 'That70sData_v60.json'
data = json.load(filename)